# ETL Pipeline for Employee Data
This notebook performs data extraction, transformation, and loading for the employee dataset.

In [2]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import numpy as np

# Load dataset
df = pd.read_csv('data.csv')
df.head()

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID
0,198,Donald,OConnell,DOCONNEL,650.507.9833,21-Jun-07,SH_CLERK,2600,-,124,50
1,199,Douglas,Grant,DGRANT,650.507.9844,13-Jan-08,SH_CLERK,2600,-,124,50
2,200,Jennifer,Whalen,JWHALEN,515.123.4444,17-Sep-03,AD_ASST,4400,-,101,10
3,201,Michael,Hartstein,MHARTSTE,515.123.5555,17-Feb-04,MK_MAN,13000,-,100,20
4,202,Pat,Fay,PFAY,603.123.6666,17-Aug-05,MK_REP,6000,-,201,20


## Step 1: Data Cleaning

In [3]:
# Replace '-' with NaN
df.replace(' - ', np.nan, inplace=True)

# Convert HIRE_DATE to datetime
df['HIRE_DATE'] = pd.to_datetime(df['HIRE_DATE'], format='%d-%b-%y', errors='coerce')

# Fill missing COMMISSION_PCT with 0
df['COMMISSION_PCT'] = pd.to_numeric(df['COMMISSION_PCT'], errors='coerce').fillna(0)

df['SALARY'] = df['SALARY'].astype(float)
df['COMMISSION_PCT'] = df['COMMISSION_PCT'].astype(float)

# Save cleaned human-readable data
df.to_csv('cleaned_data_human.csv', index=False)
df.head()


,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID
0,198,Donald,OConnell,DOCONNEL,650.507.9833,2007-06-21,SH_CLERK,2600.0,0.0,124,50
1,199,Douglas,Grant,DGRANT,650.507.9844,2008-01-13,SH_CLERK,2600.0,0.0,124,50
2,200,Jennifer,Whalen,JWHALEN,515.123.4444,2003-09-17,AD_ASST,4400.0,0.0,101,10
3,201,Michael,Hartstein,MHARTSTE,515.123.5555,2004-02-17,MK_MAN,13000.0,0.0,100,20
4,202,Pat,Fay,PFAY,603.123.6666,2005-08-17,MK_REP,6000.0,0.0,201,20


## Step 2: Prepare ML-Ready Data

In [4]:
# Select categorical and numerical columns
categorical_cols = ['JOB_ID']
numerical_cols = ['SALARY', 'COMMISSION_PCT']

# One-hot encode categorical variables
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded = encoder.fit_transform(df[categorical_cols])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(categorical_cols))

# Scale numerical features
scaler = StandardScaler()
scaled = scaler.fit_transform(df[numerical_cols])
scaled_df = pd.DataFrame(scaled, columns=numerical_cols)

# Combine scaled + encoded
ml_ready_df = pd.concat([scaled_df, encoded_df], axis=1)

# Save to CSV
ml_ready_df.to_csv('cleaned_data_ml.csv', index=False)
ml_ready_df.head()

,SALARY,COMMISSION_PCT,JOB_ID_AC_ACCOUNT,JOB_ID_AC_MGR,JOB_ID_AD_ASST,JOB_ID_AD_PRES,JOB_ID_AD_VP,JOB_ID_FI_ACCOUNT,JOB_ID_FI_MGR,JOB_ID_HR_REP,JOB_ID_IT_PROG,JOB_ID_MK_MAN,JOB_ID_MK_REP,JOB_ID_PR_REP,JOB_ID_PU_CLERK,JOB_ID_PU_MAN,JOB_ID_SH_CLERK,JOB_ID_ST_CLERK,JOB_ID_ST_MAN
0,-0.789042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,-0.789042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,-0.392574,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.501662,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.040158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
